In [ ]:
"""

重點流程：

- 掛載google drive
- 安裝套件
- 引入套件
- 建置主程序(含line_bot_api跟 webhookhandler)
- 建立主程序API介面接口(這樣才能接收用戶訊息)

* 先準備好一個按鍵模板範本(程式碼可部份參照line bot designer)
* 對用戶的訊息進行判斷，若符合條件則跳出按鍵模板範本

* 當用戶點擊按鍵模板範本後，若有postback的類型，告訴handler如何處理回傳值

- 最後三個步驟需多練習


@handler.add(XX事件) => 記憶：當handler遇到XX事件時，定義一個功能教他如何處理該事件

"""

In [ ]:
"""

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

"""

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
"""
安裝套件
"""
!pip install line-bot-sdk flask flask-ngrok

In [ ]:
"""
引用套件
"""

# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import LineBotApi, WebhookHandler

# 引用無效簽章錯誤
from linebot.exceptions import InvalidSignatureError

In [ ]:
"""
建置主程序

建置handler與 line_bot_api

"""

# 設定Server啟用細節
app = Flask(__name__, static_url_path="/material", static_folder="./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("HPNN2U0LfdttfMl7b5yCkW/h9yzuxYOGW4NYDnJuBD4qXTY/UagvGJK3OKAnGwQwgA1tYG9lPNFt92N+j33osr3nQ59os0a/bD+usRNyUumvLs0IYnHTR6FkmhCAaQ8WkcKZq6a3AJlA4tfw9jxLtgdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("e20a1f0d6418dd3bebb261a9fb9374ab")

In [ ]:
"""
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
"""
# Line Bot主程序的對話入口

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
"""
設定用戶輸入特定關鍵字後會出現的按鍵模板範本

以下設定模板消息，指定其參數細節。

"""
# ***** 此章節的核心 *****
# ***** 可搭配line bot designer *****


# 引入所需要的消息與模板消息
from linebot.models import MessageEvent, TemplateSendMessage, PostbackEvent

# 引入按鍵模板
from linebot.models.template import ButtonsTemplate


"""
建立一個按鈕多頁訊息的固定用法
buttons_template_message = TemplateSendMessage(......詳細用法看下面)

一些基本參數：

alt_text: Line簡覽視窗所出現的說明文字 (如同輪播範本的標題部分)
template: 所使用的模板
ButtonsTemplate: 按鍵模板
    thumbnail_image_url: 展示圖片
    title: 標題
    text: 說明文字
    actions: 模板行為所使用的行為(如message、uri、)
    data: 觸發postback後用戶回傳值，可以對其做商業邏輯處理



***** 其中title, text, action, data(postback才有)的設定比較重要 *****

"""

# 其他的TemplateSendMessage
# 如Python line-bot-sdk ConfirmTemplate  => template = ConfirmTemplate(.....)

#　alt_text: 初步檢視文字
# title: 標題
# text: 文字描述
# actions: 按鍵，就是json
# (actions按鍵範本最多只能有四個範本)


# buttons_template_message = TemplateSendMessage(
#     alt_text="按鍵範本測試",
#     template=ButtonsTemplate(
#         title="這是標題",
#         text="標題下方的小說明",
#         actions=[{
#               "type": "message",
#               "laber": "在範本中的點選字樣",
#               "text": "點我後顯示的字樣"
#              },
#              {
#               "type": "postback",
#               "label": "開發者，尋求教學",
#               "text": "求助專家",
#               "data": "回傳的值"
#              },
#              {
#               "type": "uri",
#               "laber": "詳可參uri scheme",
#               "uri": "https://line.me/R/nv/camera/"                  
#                  },
#                  {
#                   "type": "uri",
#                   "laber": "詳可參uri scheme",
#                   "uri": "tel://0938691851"                  
#                  }
#             ]
#     )
# )



buttons_template_message = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
        title='更多幫助',
        text='請點擊下方按鈕獲得更多幫助',
        actions=[
          {
            "type": "postback",
            "label": "企業，查找商業結合方案",
            "text": "尋找BD",
            "data": "Data1"
          },
          {
            "type": "postback",
            "label": "開發者，尋求教學",
            "text": "求助專家",
            "data": "Data2"
          },
          {
            "type": "uri",
            "label": "url scheme測試",
            "uri": "https://line.me/R/nv/camera/"
          },
          {
            "type": "uri",
            "label": "打電話",
            "uri": "tel://3345678"
          }
        ]
  )
)

In [ ]:
"""

建立一個字典，當用戶輸入特定關鍵字時，就會呼叫按鍵模板範本

"""
from linebot.models import MessageEvent, TextMessage, TextSendMessage, ImageSendMessage

template_message_dict = {
  "@more":buttons_template_message,
}

In [ ]:
"""

對用戶所發出的訊息做出判斷，並給予回應

"""


# 從linebot.models引入訊息模型，文字收取與文字寄發訊息，及影像訊息
from linebot.models import MessageEvent, TextMessage, TextSendMessage, ImageSendMessage

# 用戶發出文字消息時， 按條件內容, 回傳文字消息
# 用戶收到文字消息時的處理方式

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event): 

    if (event.message.text.find('@') != -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
        )
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="字典內查無此字，請輸入@more")
        )

In [ ]:
"""
在設定按鈕模板範本中，若設定的模板中, Action的部分有postback，

則可以進一步對postback的回傳值進行處理

=> 當用戶點擊postback類型的按鈕時，我們要告知handler怎麼做 (即告訴前台該怎麼做)

"""

# 用戶點擊button後，觸發postback event，對其回傳做相對應處理
# 當用戶點擊Postback的按鍵時，會發出PostbackEvent
# 要告訴handler收到PostbackEvent該怎麼辦

@handler.add(PostbackEvent)
def handle_post_message(event):

    user_profile = line_bot_api.get_profile(event.source.user_id)

    # 當data為data1時，則回應「有專人」
    # 若為data2時，則回應「專家」
    if (event.postback.data.find('Data1') == 0):
        with open("user_profile_business.txt", "a") as myfile:
            
            # 寫個資，然後回覆用戶
            myfile.write(json.dumps(vars(user_profile), sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(event.reply_token, TextMessage(text="請稍待，會有專人與您聯繫"))

    elif (event.postback.data.find('Data2') == 0):
        with open("user_profile_tutorial.txt", "a") as myfile:

            # 寫個資，然後回覆用戶
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(event.reply_token, TextMessage(text="請稍待，我們會派專家與您聯繫"))
    else:
        pass

In [ ]:
# 主程序運行
app.run()